# Data Initialization

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

In [0]:
Workspace.InitializeFrom(DataSource);

# Args

In [0]:
var args = new ImportArgs("CH", 2021, 3, default, default , ImportFormats.Cashflow);

# Data Node

In [0]:
var dataNodeData = (await Workspace.LoadDataNodesAsync(args))
    .Values
    .ToArray();
dataNodeData.Take(2)

In [0]:
dataNodeData


# YieldCurve

In [0]:
var lockedYieldCurves = (await Workspace.LoadLockedInYieldCurveAsync(args, dataNodeData))
    .Select(x => new { 
        DataNode = x.Key, 
        LockedIn = (x.Value.Year, x.Value.Month, x.Value.Scenario, x.Value.Currency, x.Value.Name)
        })
    .ToArray();
lockedYieldCurves.Take(2)

In [0]:
var currentYieldCurves = (await Workspace.LoadCurrentYieldCurveAsync(args, dataNodeData))
    .Values
    .SelectMany(x => x.Select(y => new { 
        Period = y.Key,
        Current = (y.Value.Year, y.Value.Month, y.Value.Scenario, y.Value.Currency, y.Value.Name)
        }))
    .ToArray();
currentYieldCurves.Take(2)

# Parameters

In [0]:
var singleDataNodeParameters = (await Workspace.LoadSingleDataNodeParametersAsync(args))
    .Values
    .SelectMany(x => x.Select(y => new { 
        Period = y.Key, 
        Parameter = (y.Value.Year, y.Value.Month, y.Value.Scenario, y.Value.DataNode, y.Value.PremiumAllocation)
        }))
    .ToArray();
singleDataNodeParameters.Take(2)

In [0]:
var interDataNodeParameters = (await Workspace.LoadInterDataNodeParametersAsync(args))
    .Values
    .SelectMany(x => x.SelectMany(y => y.Value.Select(z => new { 
        Period = y.Key, 
        //Parameter = (z.Year, z.Month, z.Scenario, z.DataNode, z.LinkedDataNode, z.ReinsuranceCoverage)
        z.Year, z.Month, z.Scenario, z.DataNode, z.LinkedDataNode, z.ReinsuranceCoverage
        })))
    .Distinct()
    .ToArray();
interDataNodeParameters.Take(2)

# Reports

In [0]:
await Report.ForObjects(interDataNodeParameters.OrderBy(x => x.DataNode))
            .WithQuerySource(Workspace)
            .ToTable()
            .ExecuteAsync()